# <font color="blue"> MBA em Ciência de Dados</font>
# <font color="blue">Programação para Ciência de Dados</font>

## <font color="blue">Pandas Parte III</font>
**Material Produzido por Luis Gustavo Nonato e Bruno Coelho**<br>
**Cemeai - ICMC/USP São Carlos**
---


__Conteúdo:__
- GroupBy
    - Agregação
    - Filtragem
    - Transformação

__Referências__ <br>
- [Pandas: powerful Python data analysis toolkit: Wes McKinney & PyData Devel. Team](https://pandas.pydata.org/pandas-docs/stable/pandas.pdf)
- [http://pandas.pydata.org/pandas-docs/stable/index.html](http://pandas.pydata.org/pandas-docs/stable/index.html)


## Introdução

Frequentemente queremos aplicar transformações e filtragens  em um conjunto de dados a fim de extrair informações relevantes e padrões contidos nos dados. Por exemplo, analisar o salário médio de um grupo de profissionais considerando o sexo e a faixa etária dos profissionais.
Muitas destas transformações podem ser realizadas por meio de uma operação denominada _MapReduce_.

_MapReduce_ é implementada no pacote <font color='blue'>Pandas</font> por meio do método <font color='blue'>groupby</font>.

Em geral, o método <font color='blue'>groupby</font> envolve 3 tarefas:
- __Split__: Divide os dados em subgrupos. Por exemplo, divide os profissionais em subgrupos de acordo com a faixa etária dos profissionais.
- __Apply__: Aplica alguma transformação, agregação ou filtragem para extrair informações de cada subgrupo. Por exemplo, pode-se calcular a média salarial em cada faixa etária dos profissionais.
- __Combine__: Combina os resultados das transformações em um DataFrame ou Série.


$$
\begin{array}{ccccc}
DataFrame & & Split & Apply & Combine\\
\begin{array}{c|c}
C1 & C2 \\ \hline
A & 0 \\ \hline
B & 5 \\ \hline
C & 10 \\ \hline
A & 5 \\ \hline
B & 5 \\ \hline
C & 10 \\ \hline
A & 10 \\ \hline
B & 5 \\ \hline
C & 10 
\end{array} &
\begin{array}{c}
\nearrow \\ \\
\rightarrow \\ \\
\searrow
\end{array} &
\begin{array}{c|c}
A & 0 \\ \hline
A & 5 \\ \hline
A & 10 \\ \\
B & 5 \\ \hline
B & 5 \\ \hline
B & 5 \\ \\
C & 10 \\ \hline
C & 10 \\ \hline
C & 10 
\end{array} &
\begin{array}{c}
\searrow\\ \\
\rightarrow \\ \\
\nearrow
\end{array} & 
\begin{array}{c|c}
A & 15 \\ \hline
B & 15 \\ \hline
C & 30 
\end{array}
\end{array}
$$

A etapa de "split" divide um conjunto de dados de acordo com algum critério, que pode ser valores das colunas do DataFrame, lista de valores externos ou até mesmo o resultado de uma função. 

Na verdade, o  <font color='blue'>pandas</font> não divide o DataFrame, mas cria uma estrutura que permite operar como se os dados estivessem divididos, evitando o uso excessivo de memória.

Portanto, é importante estar atento, pois o resultado de aplicar o método <font color='blue'>groupby</font> não é um novo DataFrame (ou Serie), mas sim um objeto do tipo `groupby`.
Para visualizar ou acessar os grupos criados pode-se realizar uma redução (que é o resultado do "apply" e "combine") ou utilizar algum método do objeto `groupby`, como por exemplo:
- <font color='blue'>first()</font>: apresenta a primeira linha de cada grupo formado
- <font color='blue'>get_group()</font>: retorna um DataFrame com o conteúdo de um grupo
- <font color='blue'>groups()</font>: retorna um dicionário onde as chaves são os rótulos dos grupos e os valores os índices das linhas onde elementos do grupo ocorrem



In [25]:
# Importando o pacote 'pandas' e 'numpy'
import pandas as pd
import numpy as np

# O comando abaixo carrega o arquivo 'tips.csv' disponível no moodle
df = pd.read_csv("tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [26]:
# Agrupando os dados de acordo com os valores da coluna "sex"

dfgb_sex = df.groupby("sex")

# Perceba que a variável 'dfgb_sex' não é um DataFrame, mas sim 
# um 'DataFrameGroupBy'
print(type(dfgb_sex))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [27]:
# visualizando a primeira linha de cada grupo
print(type(dfgb_sex.first()))  # note que ao invocar o método first() obtemos um DataFrame
print('\n primeira linha cada grupo:\n',dfgb_sex.first())  # note que os rótulos das linhas são
                                                           # os grupos obtidos

<class 'pandas.core.frame.DataFrame'>

 primeira linha cada grupo:
         total_bill   tip smoker  day    time  size
sex                                               
Female       16.99  1.01     No  Sun  Dinner     2
Male         10.34  1.66     No  Sun  Dinner     3


In [23]:
print('\n última linha cada grupo:\n',dfgb_sex.last())


 última linha cada grupo:
         total_bill   tip smoker   day    time  size
sex                                                
Female       18.78  3.00     No  Thur  Dinner     2
Male         17.82  1.75     No   Sat  Dinner     2


In [28]:
# visualizando as primeiras linhas do grupo 'Male'
print('\n',type(dfgb_sex.get_group('Male')))  # o resultado do get_group é um DataFrame
print('\n primeiras linhas do grupo "Male":\n',
      dfgb_sex.get_group('Male').head())


 <class 'pandas.core.frame.DataFrame'>

 primeiras linhas do grupo "Male":
    total_bill   tip   sex smoker  day    time  size
1       10.34  1.66  Male     No  Sun  Dinner     3
2       21.01  3.50  Male     No  Sun  Dinner     3
3       23.68  3.31  Male     No  Sun  Dinner     2
5       25.29  4.71  Male     No  Sun  Dinner     4
6        8.77  2.00  Male     No  Sun  Dinner     2


In [30]:
# o método 'groups' retorna um dicionário onde as chaves 
# são os rótulos dos grupos formados e os valores são os índices das linhas
# onde os elementos do grupo estão
print('\n',type(dfgb_sex.groups))  # o resultado de groups é um dicionário
print('\n rótulos dos grupos:\n',dfgb_sex.groups.keys())  # as chaves são os rótulos dos grupos
print('\n indices de alguns elementos do grupo "Female":\n',
      dfgb_sex.groups['Female'][0:5])


 <class 'pandas.io.formats.printing.PrettyDict'>

 rótulos dos grupos:
 dict_keys(['Female', 'Male'])

 indices de alguns elementos do grupo "Female":
 Int64Index([0, 4, 11, 14, 16], dtype='int64')


#### Agrupando de acordo com uma lista externa
O método <font color='blue'>groupby</font> pode agrupar um DataFrame de acordo com uma lista externa (que não é parte do DataFrame). Para isso, a lista deve possuir um número de elementos igual ao número de linhas do DataFrame.

In [31]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                  'key2': ['one','two','one','two','one'], 
                  'data1': np.random.uniform(low=0,high=1,size=5),
                  'data2': np.random.uniform(low=0,high=1,size=5)})
print(df)

# criando uma lista de 0 e 1 com o mesmo número de linhas do DataFrame
ls = [i for i in np.random.randint(0,2,df.shape[0])]

#lista criada
print(ls) 

  key1 key2     data1     data2
0    a  one  0.577239  0.339910
1    a  two  0.263337  0.616519
2    b  one  0.494679  0.336728
3    b  two  0.881603  0.254556
4    a  one  0.552036  0.390944
[1, 0, 0, 1, 0]


In [32]:
# Agrupando o DataFrame de acordo com a lista 'ls'
gbylist = df.groupby(ls)

print(gbylist.get_group(0))  # linhas onde o valor 0 aparece na lista
print(gbylist.get_group(1))  # linhas onde o valor 1 aparece na lista

  key1 key2     data1     data2
1    a  two  0.263337  0.616519
2    b  one  0.494679  0.336728
4    a  one  0.552036  0.390944
  key1 key2     data1     data2
0    a  one  0.577239  0.339910
3    b  two  0.881603  0.254556


#### Agrupando com múltiplos critérios (indexação hierárquica)
Quando mais que um conjunto de valores é enviado como parâmetro para o <font color='blue'>groupby</font>, o resultado é um agrupamento com índices organizados de forma hierárquica.

In [11]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','a','b','b','a','a','b','a'],
                  'key2': ['one','two','one','two','one','two','two','one'], 
                  'data1': np.random.uniform(low=0,high=1,size=8),
                  'data2': np.random.uniform(low=0,high=1,size=8)})

df

,key1,key2,data1,data2
0,a,one,0.025712,0.233743
1,a,two,0.030237,0.580148
2,b,one,0.778353,0.853350
3,b,two,0.212230,0.762083
4,a,one,0.281853,0.218681
5,a,two,0.675009,0.383792
6,b,two,0.233728,0.943275
7,a,one,0.544332,0.314939


In [12]:
# agrupando com base nos valores das colunas 'key1' e 'key2'
dfh = df.groupby(['key1','key2'])

# imprimindo o conteúdo de cada grupo com um laço for
# 'groupname' corresponde ao índice do grupo, uma tupla neste caso
# 'group' corresponde ao grupo\ própriamente dito
for groupname,group in dfh:
    print('Rotulo do Grupo: ',groupname)
    print(group,'\n')

Rotulo do Grupo:  ('a', 'one')
  key1 key2     data1     data2
0    a  one  0.025712  0.233743
4    a  one  0.281853  0.218681
7    a  one  0.544332  0.314939 

Rotulo do Grupo:  ('a', 'two')
  key1 key2     data1     data2
1    a  two  0.030237  0.580148
5    a  two  0.675009  0.383792 

Rotulo do Grupo:  ('b', 'one')
  key1 key2     data1    data2
2    b  one  0.778353  0.85335 

Rotulo do Grupo:  ('b', 'two')
  key1 key2     data1     data2
3    b  two  0.212230  0.762083
6    b  two  0.233728  0.943275 



#### Agrupando com funções
Uma função pode ser enviada como parâmetro de agrupamento para o <font color='blue'>groupby</font>. Neste caso, a função é aplicada aos rótulos das linhas do DataFrame e o resultado é utilizado como rótulo do grupo.

In [13]:
# constuindo DataFrame com rótulos de linhas e colunas
dfp = pd.DataFrame(data=np.random.randint(low=0, high=10, size=(6,5)),
               columns=['a','b','c','d','e'], 
               index=['Joe','Michel','Steve','Wes','Jim','Travis'])
dfp

,a,b,c,d,e
Joe,8,4,1,8,2
Michel,7,4,6,6,0
Steve,3,5,3,0,4
Wes,4,0,6,1,4
Jim,3,2,3,1,2
Travis,5,4,0,3,1


In [14]:
# agrupando de acordo com o número de caracteres nos rótulos das linhas
gbf = dfp.groupby(lambda x: len(x))

for name, group in gbf:
    print('Grupo: ',name)
    print(group,'\n')

Grupo:  3
     a  b  c  d  e
Joe  8  4  1  8  2
Wes  4  0  6  1  4
Jim  3  2  3  1  2 

Grupo:  5
       a  b  c  d  e
Steve  3  5  3  0  4 

Grupo:  6
        a  b  c  d  e
Michel  7  4  6  6  0
Travis  5  4  0  3  1 



# Transformações e Agregações

Em geral, temos 5 tipos de operações que podem ser aplicadas aos grupos gerados pelo <font color='blue'>groupby</font>:


- __Métodos de agregação__: Combinam várias linhas em um único valor. Exemplos incluem a média, soma e mediana de cada coluna em cada grupo.

- __Métodos de filtragem__: Retornam apenas um subconjunto dos dados originais. 

- __Métodos de tranformação__: Retornam um DataFrame com o mesmo tamanho e índices dos dados originais, mas com valores transformados com base nos grupos.

## Métodos de agregação

Pandas fornece [diversas funções](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation) estatísticas de agregação, como <font color='blue'>sum, mean, std, max, min </font>. Pode-se ainda calcular várias agregações simultaneamente com o método <font color='blue'>agg</font> (de "aggregate")

In [35]:
# Importando o pacote 'pandas'
import pandas as pd

# Carregando o arquivo 'tips.csv'
df = pd.read_csv("tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [173]:
# Obtendo a maior gorjeta dos grupos 'Male' e 'Female'
print("Gorjeta máxima")

# Agrupamos por sexo, depois selecionamos a coluna "tip" (gorjeta)
# e pegamos o máximo dela.
print(df.groupby(["sex"])["tip"].max())


Gorjeta máxima
sex
Female     6.5
Male      10.0
Name: tip, dtype: float64


In [36]:
# Pode-se calcular várias agregações simultaneamente via método "agg"
print("Média e Desvio Padrão das gorjetas em cada grupo:")
df.groupby(["sex"])["tip"].agg(["mean","std"])

Média e Desvio Padrão das gorjetas em cada grupo:


,mean,std
sex,,
Female,2.833448,1.159495
Male,3.089618,1.489102


**Importante**: O resultado de uma agregação, transformação ou filtragem  é um novo DataFrame onde os rótulos das linhas são os valores utilizados para realizar o agrupamento. 
Caso queiramos um DataFrame com linhas indexadas com valores $0,1,...$, devemos empregar o método <font color='blue'>reset_index</font>.

In [4]:
# Usando as colunas de agregação como índices
result = df.groupby(["sex"])["tip"].agg(["mean", "std"])
print(result,'\n')
print("Rotulos dos grupos:\n", result.index)
print("\nRotulos das colunas:\n", result.columns)

            mean       std
sex                       
Female  2.833448  1.159495
Male    3.089618  1.489102 

Rotulos dos grupos:
 Index(['Female', 'Male'], dtype='object', name='sex')

Rotulos das colunas:
 Index(['mean', 'std'], dtype='object')


In [5]:
# Reindexando com reset_index(), os rótulos se tornam uma nova coluna
result = df.groupby(["sex"])["tip"].agg(["mean", "std"]).reset_index()
print(result,'\n')
print("Rotulos dos grupos:\n", result.index)
print("\nRotulos das colunas:\n", result.columns)

      sex      mean       std
0  Female  2.833448  1.159495
1    Male  3.089618  1.489102 

Rotulos dos grupos:
 RangeIndex(start=0, stop=2, step=1)

Rotulos das colunas:
 Index(['sex', 'mean', 'std'], dtype='object')


### Pode-se aplicar agregações diferentes para cada coluna com o uso de dicionário.

In [37]:
print(df.head())
# calculando a média das gorjetas, 
# o valor máximo das refeições 
# e o dia da semana mais frequente (moda) de cada grupo 'Male' e 'Female'
dfm = df.groupby(["sex"]).agg({"tip": "mean","total_bill": "max","day": lambda x: x.mode()})

print('\n',dfm.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4

              tip  total_bill   day
sex                               
Female  2.833448       44.30  Thur
Male    3.089618       50.81   Sat


In [38]:
# Pode-se renomear as colunas do DataFrame gerado durante a agregação
# renomeando colunas com agg
print(df.groupby(["sex"]).agg(mean_tip=("tip", "mean"),
                              max_bill=("total_bill", "max"),
                              most_freq_day=("day", lambda x: x.mode())))

        mean_tip  max_bill most_freq_day
sex                                     
Female  2.833448     44.30          Thur
Male    3.089618     50.81           Sat


## Métodos de filtragem

O método mais comum de filtragem é o <font color='blue'>filter </font>que retorna os elementos do grupo que satisfaz uma condição dada. 

In [39]:
import numpy as np
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','c','b','c','a','a','b','a','c','b'],
                  'data0': np.random.uniform(low=0,high=1,size=10),
                  'data1': np.random.uniform(low=0,high=1,size=10),
                  'data2': np.random.uniform(low=0,high=1,size=10)})

df

,key1,data0,data1,data2
0,a,0.590898,0.663302,0.364411
1,c,0.063587,0.718407,0.881627
2,b,0.233773,0.341686,0.641104
3,c,0.079769,0.844161,0.772516
4,a,0.901859,0.775796,0.929725
5,a,0.770314,0.664571,0.062814
6,b,0.968807,0.819561,0.692521
7,a,0.302924,0.580932,0.770864
8,c,0.698425,0.001452,0.678423
9,b,0.292106,0.439569,0.776854


In [16]:
# média geral da coluna 'data1'
data1_mean = df['data1'].mean()
print('media da coluna "data1"\n',data1_mean)

# calculando a média de cada coluna em cada grupo dado pela coluna 'key1'
print('\nmedia de cada grupo\n',df.groupby('key1').mean())

# filtrando os grupos cuja média da coluna 'data1' é maior que a média 
# geral da coluna 'data1'
print('\n grupos cuja média da coluna "data1" é maior que a média geral de "data1"\n')
print(df.groupby('key1').filter(lambda x : x['data1'].mean() > data1_mean))

media da coluna "data1"
 0.5641340885448626

media de cada grupo
          data0     data1     data2
key1                              
a     0.440002  0.305379  0.228003
b     0.470074  0.844564  0.653952
c     0.478482  0.628710  0.854560

 grupos cuja média da coluna "data1" é maior que a média geral de "data1"

  key1     data0     data1     data2
1    c  0.694699  0.966926  0.940898
2    b  0.516477  0.904974  0.702352
3    c  0.526632  0.284643  0.892902
6    b  0.083109  0.930837  0.914147
8    c  0.214116  0.634562  0.729880
9    b  0.810637  0.697881  0.345358


## Métodos de transformação

O método <font color='blue'>transform</font> retorna um DataFrame com o **mesmo número de linhas** que o DataFrame original. A transformação é realizada em cada elemento de cada grupo.

In [40]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','c','b','c','a','a','b','a','c','b'],
                  'data0': np.random.uniform(low=0,high=1,size=10),
                  'data1': np.random.uniform(low=0,high=1,size=10),
                  'data2': np.random.uniform(low=0,high=1,size=10)})

df

,key1,data0,data1,data2
0,a,0.013786,0.909521,0.854145
1,c,0.777704,0.856443,0.436721
2,b,0.343789,0.298247,0.597425
3,c,0.943504,0.342788,0.660669
4,a,0.006209,0.564335,0.292543
5,a,0.020974,0.097755,0.315373
6,b,0.382749,0.156379,0.310163
7,a,0.401390,0.043713,0.323816
8,c,0.880574,0.387380,0.527184
9,b,0.757488,0.959226,0.849573


In [18]:
# Calcula a diferença entre a média do grupo e 
# o elemento do grupo (em cada coluna)
print('média de cada grupo\n',df.groupby("key1").mean())

df[['d1','d2','d3']] = df.groupby("key1").transform(lambda x: x-x.mean())
print('\n',df)

média de cada grupo
          data0     data1     data2
key1                              
a     0.475076  0.581153  0.428277
b     0.339218  0.502673  0.607236
c     0.500801  0.561973  0.513310

   key1     data0     data1     data2        d1        d2        d3
0    a  0.752687  0.703462  0.816468  0.277611  0.122309  0.388191
1    c  0.711394  0.237431  0.616238  0.210592 -0.324542  0.102928
2    b  0.138789  0.629330  0.752405 -0.200429  0.126657  0.145169
3    c  0.390139  0.659471  0.717868 -0.110662  0.097498  0.204558
4    a  0.118797  0.835410  0.392499 -0.356279  0.254257 -0.035778
5    a  0.811809  0.079336  0.360627  0.336733 -0.501817 -0.067650
6    b  0.752516  0.294782  0.900429  0.413298 -0.207890  0.293193
7    a  0.217012  0.706405  0.143515 -0.258064  0.125252 -0.284762
8    c  0.400872  0.789017  0.205824 -0.099930  0.227044 -0.307486
9    b  0.126350  0.583906  0.168874 -0.212868  0.081233 -0.438362


O método <font color='blue'>apply</font> aplica uma função em cada grupo e retorna um DataFrame com o resultado da função em cada grupo. Os rótulos das linhas do DataFrame gerado são os identificadores dos grupos. O método <font color='blue'>apply</font> é o mais versátil dos métodos de transformação.

In [41]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','c','b','c','a','a','b','a','c','b'],
                  'data0': np.random.uniform(low=0,high=1,size=10),
                  'data1': np.random.uniform(low=0,high=1,size=10),
                  'data2': np.random.uniform(low=0,high=1,size=10)})

df

,key1,data0,data1,data2
0,a,0.999393,0.134854,0.340989
1,c,0.117748,0.744118,0.230955
2,b,0.495444,0.262164,0.952003
3,c,0.865078,0.721211,0.742631
4,a,0.670826,0.189563,0.009071
5,a,0.152570,0.527849,0.239987
6,b,0.061284,0.628577,0.639319
7,a,0.293411,0.791821,0.135858
8,c,0.496162,0.961006,0.721076
9,b,0.561485,0.978343,0.845561


In [42]:
for groupname, group in df.groupby("key1"):
    print(groupname)
    print(group)

a
  key1     data0     data1     data2
0    a  0.999393  0.134854  0.340989
4    a  0.670826  0.189563  0.009071
5    a  0.152570  0.527849  0.239987
7    a  0.293411  0.791821  0.135858
b
  key1     data0     data1     data2
2    b  0.495444  0.262164  0.952003
6    b  0.061284  0.628577  0.639319
9    b  0.561485  0.978343  0.845561
c
  key1     data0     data1     data2
1    c  0.117748  0.744118  0.230955
3    c  0.865078  0.721211  0.742631
8    c  0.496162  0.961006  0.721076


In [43]:
# agrupando pela coluna 'key1' e calculando a 
# diferença entre o maior e o menor valor de cada coluna em cada grupo
# e retornando a menor diferença

# diferença entre o maior e menor valor em cada coluna em cada grupo
print(df.groupby('key1').apply(lambda x: (x.max()-x.min())))

# menor diferença em cada grupo
max_min = df.groupby('key1').apply(lambda x: (x.max()-x.min()).sort_values()[0])
print(max_min)

         data0     data1     data2
key1                              
a     0.846823  0.656967  0.331919
b     0.500201  0.716179  0.312685
c     0.747330  0.239795  0.511676
key1
a    0.331919
b    0.312685
c    0.239795
dtype: float64


**Comentário: MapReduce**

Embora frequentemente chamamos qualquer operação split-apply-combine de _MapReduce_, originalmente esse nome foi dado ao modelo de programação desenvolvido pela Apache.

Sua funcionalidade é a mesma do _GroupBy_ do Pandas, porém foi implementado para operar em grandes volumes de dados utilizando algoritmos distribuidos em uma arquitetura de computação paralela (cluster de computadores), garantindo que cada máquina opere sobre um subconjunto específico dos dados, sendo o padrão para tecnologias de Big Data como [Hadoop](https://hadoop.apache.org/)
